<a href="https://colab.research.google.com/github/EAakiyama3104/python_lecture/blob/master/%5BPython%E8%AC%9B%E5%BA%A7%5D%E7%AC%AC4%E5%9B%9ESpotify_API%E3%82%92%E7%94%A8%E3%81%84%E3%81%9F%E3%83%87%E3%83%BC%E3%82%BF%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Spotify API とは

* Spotify 上の音楽データを提供しているWeb API
* Spotify上のアーティスト情報や曲の情報などを取得できる

詳細: https://developer.spotify.com/documentation/web-api/reference/

Spotify API の特徴

![代替テキスト](https://cdn-ssl-devio-img.classmethod.jp/wp-content/uploads/2017/12/spotify-api-overview.png)

* Spotify URIやSpotify IDなど、Spotify特有のパラメータ情報をリクエスト、レスポンス時に扱うことがあります。
* レート制限に引っかかるとステータスコード429が返却される。これが返ってきた場合は、Retry-Afterヘッダーの値を確認し、そこに記載されている秒の間はAPIをコールすることができない。

* レスポンスデータは、JSON Object形式で取得します。

* タイムスタンプはUTC zero offset形式（YYYY-MM-DDTHH:MM:SSZ）

* 一部のAPIではページネーションをサポート（offset、limitパラメータで指定）

* 多くのAPIでは、クライアント側でレスポンスデータのキャッシュを行うためのヘッダー情報が付与される。

* レスポンスステータスコードは、RFC 2616 と RFC 6585に則って2xx、3xx、4xx、5xxを返す。

* レスポンスデータのエラー表現には、下記2種類のフォーマットを使用する。

  * Authentication Error Object
  * Regular Error Object
*APIをコールするにはOAuthアクセストークンが必要。

認証方法の種類

![代替テキスト](https://cdn-ssl-devio-img.classmethod.jp/wp-content/uploads/2017/12/Obtaining-Authorization.png)

* App Authorization
  * システム（Spotify）が、クライアントアプリケーションに、Spotify Platform (APIs, SDKs and Widgets)へアクセスすることを許可します。
* User Authorization
  * エンドユーザー（Spotifyのエンドユーザー）が、クライアントアプリケーションに、エンドユーザーが所有するデータへアクセスすることを許可します。

アプリが認証の許可を得るための3つの方法


* Refreshable user authorization: Authorization Code
  * このフローでは、エンドユーザーはクライアントアプリケーションがリソースへアクセスすることを1度だけ許可します。そのため長期間稼働させるアプリケーションに適しています。 アプリケーションにはリフレッシュ可能なアクセストークンを提供されます。ちなみにトークン交換には秘密鍵の送信が含まれるため、ブラウザやモバイルアプリなどのクライアントからではなく、バックエンドサービスなどの安全な場所でこれを実行します。
* Temporary user authorization: Implicit Grant
  * このフローは、JavaScriptを使用するため、リソース所有者のブラウザで実行されるクライアントアプリケーション向けです。サーバー側のコードを使用する必要はありません。リクエストのレート制限は改善されていますが、リフレッシュトークンは提供されていません。このフローはRFC-6749で定義されています。
* Refreshable app authorization: Client Credentials Flow
  * このフローは、サーバー間認証で使用されます。エンドユーザー情報にアクセスしないエンドポイントのみにアクセスが可能です。
  * このセミナーではこの認証方法を使用

## アカウントを作る

1. https://www.spotify.com/ にアクセスしてアカウントを作る。
2. https://developer.spotify.com/my-applications に行ってログイン
3. Create an Application に Application Name と Description を入れる
4. Are you developing a commercial integration? と聞かれたら、この講座では NO を選択
5. チェックボックスをオンにして Submit
6. Client ID と Client Secret メモしておく。APIを呼ぶ際に使います。

## Spotipyのインストール

Spotipy とは、Spotify APIを呼ぶためのPythonライブラリ

* HTTPリクエストを送る操作をライブラリ内で行っているため、リクエストを意識せずに使うことができる
* pip install spotipy でインストール

In [0]:
!pip install spotipy

In [0]:
import spotipy

spotipy.Spotify クラスがAPIへ様々なリクエストを送る

https://spotipy.readthedocs.io/en/latest/#module-spotipy.client

In [0]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

CLIENT_ID = ""
CLIENT_SECRET = ""

# 認証情報をセット
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
# 認証情報を元に、APIクライアントを作成
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

プレイリストを取得

公式リファレンス: https://developer.spotify.com/documentation/web-api/reference/playlists/get-list-users-playlists/

Spotipyリファレンス: https://spotipy.readthedocs.io/en/latest/#spotipy.client.Spotify.user_playlist

例: プレイリストのタイトルとURLを取得する

In [0]:
# Spotify 公式のプレイリストから2つ取得
playlists = sp.user_playlists('spotify', limit=2)
playlists

pprint を用いるとよりきれいにデータを表示できる

https://docs.python.org/ja/3/library/pprint.html

In [0]:
from pprint import pprint

In [0]:
pprint(playlists)

In [0]:
pprint(playlists['items'])

In [0]:
pprint(playlists['items'][0])

In [0]:
# タイトルを取得
pprint(playlists['items'][0]['name'])

In [0]:
# URLを取得
pprint(playlists['items'][0]['external_urls']['spotify'])

In [0]:
# 複数のタイトルとURLを取得
for playlist in playlists['items']:
  # playlists['items'][0] を playlist で置き換える
  pprint('タイトル: ' + playlist['name'])
  pprint('URL: ' + playlist['external_urls']['spotify'])

In [0]:
birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'
spotify = spotipy.Spotify()

results = sp.artist_albums(birdy_uri, album_type='album')
albums = results['items']
while results['next']:
  results = sp.next(results)
  albums.extend(results['items'])

for album in albums:
  print(album['name'])

必要なライブラリを読み込み

In [0]:
import pandas as pd
import seaborn as sns

ジャンル情報を取得

In [0]:
sp.recommendation_genre_seeds()

In [0]:
pd.DataFrame(sp.recommendation_genre_seeds())

In [0]:
genres_df = pd.DataFrame(sp.recommendation_genre_seeds())

特定のジャンルの曲を取得

/search API を用いる

* 公式リファレンス: https://developer.spotify.com/documentation/web-api/reference/search/search/
* spotipy のリファレンス: https://spotipy.readthedocs.io/en/latest/#spotipy.client.Spotify.search

In [0]:
# ジャンルで検索
search_genre = sp.search(q='genre:acoustic', type='track', offset=0, limit=1)
pprint(search_genre)

検索結果の例

```
{'tracks': {'href': 'https://api.spotify.com/v1/search?query=genre%3Aacoustic&type=track&offset=0&limit=1',
            'items': [{'album': {'album_type': 'single',
                                 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5D3muNJhYYunbRkh3FKgX0'},
                                              'href': 'https://api.spotify.com/v1/artists/5D3muNJhYYunbRkh3FKgX0',
                                              'id': '5D3muNJhYYunbRkh3FKgX0',
                                              'name': 'Chord Overstreet',
                                              'type': 'artist',
                                              'uri': 'spotify:artist:5D3muNJhYYunbRkh3FKgX0'}],
                                 'available_markets': ['AD',
                                                       'AE',
                                                       'AR',
                                                       'AT',
                                                       'AU',
                                                       'BE',
                                                       'BG',
                                                       'BH',
                                                       'BO',
                                                       'BR',
                                                       'CA',
                                                       'CH',
                                                       'CL',
                                                       'CO',
                                                       'CR',
                                                       'CY',
                                                       'CZ',
                                                       'DE',
                                                       'DK',
                                                       'DO',
                                                       'DZ',
                                                       'EC',
                                                       'EE',
                                                       'EG',
                                                       'ES',
                                                       'FI',
                                                       'FR',
                                                       'GB',
                                                       'GR',
                                                       'GT',
                                                       'HK',
                                                       'HN',
                                                       'HU',
                                                       'ID',
                                                       'IE',
                                                       'IL',
                                                       'IN',
                                                       'IS',
                                                       'IT',
                                                       'JO',
                                                       'JP',
                                                       'KW',
                                                       'LB',
                                                       'LI',
                                                       'LT',
                                                       'LU',
                                                       'LV',
                                                       'MA',
                                                       'MC',
                                                       'MT',
                                                       'MX',
                                                       'MY',
                                                       'NI',
                                                       'NL',
                                                       'NO',
                                                       'NZ',
                                                       'OM',
                                                       'PA',
                                                       'PE',
                                                       'PH',
                                                       'PL',
                                                       'PS',
                                                       'PT',
                                                       'PY',
                                                       'QA',
                                                       'RO',
                                                       'SA',
                                                       'SE',
                                                       'SG',
                                                       'SK',
                                                       'SV',
                                                       'TH',
                                                       'TN',
                                                       'TR',
                                                       'TW',
                                                       'US',
                                                       'UY',
                                                       'VN',
                                                       'ZA'],
                                 'external_urls': {'spotify': 'https://open.spotify.com/album/2EfmyRWheMtmVTCIsptsLi'},
                                 'href': 'https://api.spotify.com/v1/albums/2EfmyRWheMtmVTCIsptsLi',
                                 'id': '2EfmyRWheMtmVTCIsptsLi',
                                 'images': [{'height': 640,
                                             'url': 'https://i.scdn.co/image/bbf0818eba3b1fd85a4b737b8464fac1ff9609a8',
                                             'width': 640},
                                            {'height': 300,
                                             'url': 'https://i.scdn.co/image/cea87b9099170adf673a610084f187f60c745fc3',
                                             'width': 300},
                                            {'height': 64,
                                             'url': 'https://i.scdn.co/image/32e22b621cefd4cca9afbdfc7fa86e7e901a8250',
                                             'width': 64}],
                                 'name': 'Hold On',
                                 'release_date': '2017-02-03',
                                 'release_date_precision': 'day',
                                 'total_tracks': 1,
                                 'type': 'album',
                                 'uri': 'spotify:album:2EfmyRWheMtmVTCIsptsLi'},
                       'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5D3muNJhYYunbRkh3FKgX0'},
                                    'href': 'https://api.spotify.com/v1/artists/5D3muNJhYYunbRkh3FKgX0',
                                    'id': '5D3muNJhYYunbRkh3FKgX0',
                                    'name': 'Chord Overstreet',
                                    'type': 'artist',
                                    'uri': 'spotify:artist:5D3muNJhYYunbRkh3FKgX0'}],
                       'available_markets': ['AD',
                                             'AE',
                                             'AR',
                                             'AT',
                                             'AU',
                                             'BE',
                                             'BG',
                                             'BH',
                                             'BO',
                                             'BR',
                                             'CA',
                                             'CH',
                                             'CL',
                                             'CO',
                                             'CR',
                                             'CY',
                                             'CZ',
                                             'DE',
                                             'DK',
                                             'DO',
                                             'DZ',
                                             'EC',
                                             'EE',
                                             'EG',
                                             'ES',
                                             'FI',
                                             'FR',
                                             'GB',
                                             'GR',
                                             'GT',
                                             'HK',
                                             'HN',
                                             'HU',
                                             'ID',
                                             'IE',
                                             'IL',
                                             'IN',
                                             'IS',
                                             'IT',
                                             'JO',
                                             'JP',
                                             'KW',
                                             'LB',
                                             'LI',
                                             'LT',
                                             'LU',
                                             'LV',
                                             'MA',
                                             'MC',
                                             'MT',
                                             'MX',
                                             'MY',
                                             'NI',
                                             'NL',
                                             'NO',
                                             'NZ',
                                             'OM',
                                             'PA',
                                             'PE',
                                             'PH',
                                             'PL',
                                             'PS',
                                             'PT',
                                             'PY',
                                             'QA',
                                             'RO',
                                             'SA',
                                             'SE',
                                             'SG',
                                             'SK',
                                             'SV',
                                             'TH',
                                             'TN',
                                             'TR',
                                             'TW',
                                             'US',
                                             'UY',
                                             'VN',
                                             'ZA'],
                       'disc_number': 1,
                       'duration_ms': 198853,
                       'explicit': False,
                       'external_ids': {'isrc': 'USUM71615568'},
                       'external_urls': {'spotify': 'https://open.spotify.com/track/5vjLSffimiIP26QG5WcN2K'},
                       'href': 'https://api.spotify.com/v1/tracks/5vjLSffimiIP26QG5WcN2K',
                       'id': '5vjLSffimiIP26QG5WcN2K',
                       'is_local': False,
                       'name': 'Hold On',
                       'popularity': 79,
                       'preview_url': None,
                       'track_number': 1,
                       'type': 'track',
                       'uri': 'spotify:track:5vjLSffimiIP26QG5WcN2K'}],
            'limit': 1,
            'next': 'https://api.spotify.com/v1/search?query=genre%3Aacoustic&type=track&offset=1&limit=1',
            'offset': 0,
            'previous': None,
            'total': 64838}}
```



## 問題: 検索結果から曲名を取得して表示してください。

## 解答

In [0]:
# tracks > items から検索結果を取得
pprint(search_genre['tracks']['items'])

In [0]:
# 最初のトラックを取得
pprint(search_genre['tracks']['items'][0])

In [0]:
# 曲名を取得
pprint(search_genre['tracks']['items'][0]['name'])

In [0]:
# 曲のIDを取得
pprint(search_genre['tracks']['items'][0]['id'])

## トラック ID から曲の特徴を取得

/tracks/get-several-audio-features API を用いる

spotipy では audio_features メソッド

* https://developer.spotify.com/documentation/web-api/reference/tracks/get-several-audio-features/
* https://spotipy.readthedocs.io/en/latest/#spotipy.client.Spotify.audio_features

danceability

danceability(踊りやすさ)は、テンポ、リズムの安定性、ビートの強さ、全体的な規則性などの音楽要素の組み合わせに基づいて、ダンスのための曲であるかを示します。0.0の値は、最も踊りずらいことを、1.0は、最も踊りやすいことを示します。

energy

エネルギーは、0.0から1.0の指標で、強度およびアクティブ度を表します。一般的には、エネルギッシュなトラックは、速く、音が大きく、騒々しい感じがします。例えば、デスメタルは高いエネルギーを持っていますが、バッハのプレリュードは低い値になります。この属性に寄与する知覚的属性は、ダイナミックレンジ、聴覚が感じる音量、音色、開始時点のレート、および一般的なエントロピーを含みます。

loudness

ラウドネスは、曲の全体的な音量をデシベル（dB）で表したものです。値は曲全体で平均化され、他の曲との相対的なラウドネスを比較するのに役立ちます。値の典型的な範囲は-60から0dbです。

speechiness

スピーチは、曲の中で話された単語の存在を検出します。録音（例えば、トークショー、オーディオブック、詩）のように、音声が占める割合が大きくなるほど、値は1.0に近くなります。0.66を超える値は、ほぼ完全に発声された単語で構成されている曲を表します。0.33と0.66の間の値は、ラップ音楽などのセクションまたはレイヤーのいずれかで、音楽とスピーチの両方を含む可能性がある曲を表します。0.33未満の値は、音楽やその他の非音声のような曲を表す可能性がかなり高いことを示します。

acousticness

曲がアコースティックかどうかを示す0.0から1.0の指標です。 1.0は曲がアコースティックであるということが高いということを意味します。

instrumentalness

曲にボーカルがないかどうかを予測します。この指標では、 “オー(Ooh)”とか “アー(aah)”の音は楽器の出した音として扱われます。ラップや話し言葉はボーカルとして扱われます。インストゥルメンタルネスの値が1.0に近いほど、曲にはボーカル・コンテンツが含まれていない可能性が高くなります。 0.5を超える値は、インストゥルメンタルの曲が通常示す値ですが、値が1.0に近づくほど信頼度が高くなります。

liveness

録音中に聴衆が存在したかを検出します。この値が高いほど、曲がライブで実行された可能性が高くなります。値が0.8を超えると、曲がライブである可能性が高くなります。

valence

曲が伝える音楽のポジティブ性を表す0.0から1.0の尺度。この指数の高い値の曲はより陽性（例えば、幸せ、陽気、陶酔）であり、低い指数の曲はより陰性となります（例えば、悲しい、落ち込んだ、怒る）。

tempo

曲の全体的な推定テンポ。1分あたりのビート(BPM)。音楽用語では、テンポは、ある曲のスピードまたはペースであり、平均ビート期間から導出されます。


以下の記事より引用
https://exploratory.io/note/2ac8ae888097/SpotifyJRockRockHard-Rock-7183595689717906

In [0]:
# 曲の特徴を取得
pprint(sp.audio_features(tracks=[search_genre['tracks']['items'][0]['id']]))



```
[{'acousticness': 0.469,
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5vjLSffimiIP26QG5WcN2K',
  'danceability': 0.618,
  'duration_ms': 198853,
  'energy': 0.443,
  'id': '5vjLSffimiIP26QG5WcN2K',
  'instrumentalness': 0,
  'key': 2,
  'liveness': 0.0829,
  'loudness': -9.681,
  'mode': 1,
  'speechiness': 0.0526,
  'tempo': 119.949,
  'time_signature': 4,
  'track_href': 'https://api.spotify.com/v1/tracks/5vjLSffimiIP26QG5WcN2K',
  'type': 'audio_features',
  'uri': 'spotify:track:5vjLSffimiIP26QG5WcN2K',
  'valence': 0.167}]
```



## ジャンル毎に100曲を取得

In [0]:
# キーを定数に保存
ACOUSTICNESS = 'acousticness'
DANCEABILITY = 'danceability'
INSTRUMENTALNESS = 'instrumentalness'
LIVENESS = 'liveness'
LOUDNESS = 'loudness'
SPEECHINESS = 'speechiness'
TEMPO = 'tempo'
VALENCE = 'valence'

In [0]:
# データ格納用のdictionaryを用意
tracks = {}
tracks['id'] = []
tracks['name'] = []
tracks['genre'] = []
tracks[ACOUSTICNESS] = []
tracks[DANCEABILITY] = []
tracks[INSTRUMENTALNESS] = []
tracks[LIVENESS] = []
tracks[LOUDNESS] = []
tracks[SPEECHINESS] = []
tracks[TEMPO] = []
tracks[VALENCE] = []

In [0]:
# ジャンル毎に1000個のトラックを取得し、tracks に保存
TRACK_COUNT_PER_GENRE = 100 # ジャンル毎の曲数
TRACK_COUNT_PER_SEARCH = 50 # 1回の検索で取得する曲数
SEARCH_COUNT = int(TRACK_COUNT_PER_GENRE / TRACK_COUNT_PER_SEARCH) # 検索を行う回数 - この場合は2回になる

for genre in genres_df['genres']:
  for search_count in range(SEARCH_COUNT):
    query = 'genre:' + genre # ジャンルで検索するようにクエリを作成
    offset = TRACK_COUNT_PER_SEARCH * search_count # offsetで検索結果をページのようにズラしていく
    search_results = sp.search(q=query, type='track', offset=offset, limit=TRACK_COUNT_PER_SEARCH) # 検索を行う
    # items が空の場合は、そのジャンルの取得を終了
    if not search_results['tracks']['items']: 
      break
    # 取得したトラックのID
    track_ids = []
    # 各トラックに対して id などを track_ids, tracks に保存
    for track in search_results['tracks']['items']:
      track_ids.append(track.get('id'))
      tracks['id'].append(track.get('id'))
      tracks['name'].append(track.get('name'))
      tracks['genre'].append(genre)
    # 曲の特徴を取得
    audio_features = sp.audio_features(tracks=track_ids)
    # 各トラックに対して、曲の特徴を tracks に保存
    for audio_feature in audio_features:
      tracks[ACOUSTICNESS].append(audio_feature.get(ACOUSTICNESS))
      tracks[DANCEABILITY].append(audio_feature.get(DANCEABILITY))
      tracks[INSTRUMENTALNESS].append(audio_feature.get(INSTRUMENTALNESS))
      tracks[LIVENESS].append(audio_feature.get(LIVENESS))
      tracks[LOUDNESS].append(audio_feature.get(LOUDNESS))
      tracks[SPEECHINESS].append(audio_feature.get(SPEECHINESS))
      tracks[TEMPO].append(audio_feature.get(TEMPO))
      tracks[VALENCE].append(audio_feature.get(VALENCE))

In [0]:
for key in tracks:
  print(f'{key}: {len(tracks[key])}')

In [0]:
tracks_df = pd.DataFrame(tracks)

In [0]:
tracks_df

ジャンルの数と曲数が合わない理由

どのトラックにも該当しないジャンルがある

In [0]:
tracks_df['genre'].unique()

In [0]:
genres_df['genres'].unique()

## 課題
tracks_df のデータを seaborn などで可視化してグラフに表示してください。